In [1]:
!pip install -q -U accelerate peft bitsandbytes transformers trl datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 113.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 95.4 MB/s eta 0:00:00


In [ ]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaConfig
from peft import LoraConfig
from trl import SFTTrainer

In [3]:
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

Prerequisites

In [4]:
model_id = "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ["HF_TOKEN"])

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"":0},
    token=os.environ["HF_TOKEN"]
)

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [6]:
text = "Quote : Imagine is more"
device = "cuda:0"
input = tokenizer(text, return_tensors="pt").to(device)

output = model.generate(**input, max_new_tokens=30)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Quote : Imagine is more than just a place to live. It’s a place to call home.

Imagine is a new residential development by Imagine Group. The project is


In [7]:
os.environ["WANDB_DISABLED"] = "false"

In [8]:
lora_config = LoraConfig(
    r = 8,
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    task_type = 'CAUSAL_LM'
)

In [9]:
from datasets import load_dataset

data = load_dataset("Abirate/english_quotes")

README.md: 0.00B [00:00, ?B/s]

quotes.jsonl:   0%|          | 0.00/647k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [13]:
data["train"]["quote"][1]

"“I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.”"

In [11]:
def formatting_func(example):
    return f"Quote: {example['quote']}\nAuthor: {example['author']}"

In [15]:
print(formatting_func(data["train"][1]))

Quote: “I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.”
Author: Marilyn Monroe


In [16]:
data['train']

Dataset({
    features: ['quote', 'author', 'tags'],
    num_rows: 2508
})

In [19]:
trainer = SFTTrainer(
    model = model,
    train_dataset = data['train'],
    args = transformers.TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 2,
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = True,
        logging_steps = 1,
        output_dir = 'outputs',
        optim = "paged_adamw_8bit"
    ),
    peft_config = lora_config,
    formatting_func = formatting_func,
)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [20]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mohsin416 (mohsin416-mohsin) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
1,2.655000
2,2.092900
3,2.581100
4,2.823000
5,2.384200
6,2.444400
7,2.795600
8,2.064300
9,2.892600
10,2.223200


TrainOutput(global_step=100, training_loss=1.8122899252176286, metrics={'train_runtime': 181.9872, 'train_samples_per_second': 2.198, 'train_steps_per_second': 0.549, 'total_flos': 235740440186880.0, 'train_loss': 1.8122899252176286, 'epoch': 0.1594896331738437})

In [21]:
text = "Quote : A woman is like a tea bag"
device = "cuda:0"
input = tokenizer(text, return_tensors="pt").to(device)

output = model.generate(**input, max_new_tokens=20)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Quote : A woman is like a tea bag. You never know how strong she is until she gets in hot water.
Author : Eleanor Roosevelt
